# Fashion MNIST Project
classify Fashion-MNIST dataset using convolutional neural networks.

In [ ]:
!pip install torch
!pip install torchvision
!pip install matplotlib

In [2]:
# PyTorch Modules you need for this lab

from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
torch.manual_seed(0)

In [3]:
# Other non-PyTorch Modules

from matplotlib.pyplot import imshow
import matplotlib.pylab as plt

from PIL import Image

In [4]:
def show_data(data_sample):
    plt.imshow(data_sample[0].numpy().reshape(IMAGE_SIZE, IMAGE_SIZE), cmap='gray')
    plt.title('y = '+ str(data_sample[1]))

In [9]:
IMAGE_SIZE = 16

# transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
# transforms.ToTensor()#
# composed = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.ToTensor()])


composed = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor()         
])

# Load the training dataset with the transform applied
train_dataset = dsets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=composed
)

# Load the test dataset with the same transform
test_dataset = dsets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=composed
)